In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)+(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

In [3]:
who

Interactive namespace is empty.


In [4]:
import numpy as np
import pandas as pd
import scipy as sp
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
w1 = 0.31
w2 = 0.81
a1r = -0.51
a1i = 0.15
a2r = 1.84
a2i = -3.22
b1r = 0.152
b1i = -2.821
b2r = -0.818
b2i = -0.775
def fun(x):
    return np.array([-w1*x[1]  + (a1r*x[0] - a1i*x[1])*(x[0]**2 + x[1]**2) + (a2r*x[0] - a2i*x[1])*(x[2]**2 + x[3]**2) + (x[0]+x[1]+x[2]+x[3])**5,
                      w1*x[0]  + (a1r*x[1] + a1i*x[0])*(x[0]**2 + x[1]**2) + (a2r*x[1] + a2i*x[0])*(x[2]**2 + x[3]**2) + (x[0]+x[1]+x[2]+x[3])**5,
                     -w2*x[3]  + (b1r*x[2] - b1i*x[3])*(x[2]**2 + x[3]**2) + (b2r*x[2] - b2i*x[3])*(x[0]**2 + x[1]**2) + (x[0]+x[1]+x[2]+x[3])**5,
                      w2*x[2]  + (b1r*x[3] + b1i*x[2])*(x[2]**2 + x[3]**2) + (b2r*x[3] + b2i*x[2])*(x[0]**2 + x[1]**2) + (x[0]+x[1]+x[2]+x[3])**5
                    ])

def f1(x): 
    return np.array([-w1*x[1] + (a1r*x[0] - a1i*x[1])*(x[0]**2 + x[1]**2) + (a2r*x[0] - a2i*x[1])*(x[2]**2 + x[3]**2) + (x[0]+x[1]+x[2]+x[3])**5])
def f2(x):
    return np.array([ w1*x[0] + (a1r*x[1] + a1i*x[0])*(x[0]**2 + x[1]**2) + (a2r*x[1] + a2i*x[0])*(x[2]**2 + x[3]**2) + (x[0]+x[1]+x[2]+x[3])**5])
def f3(x):
    return np.array([-w2*x[0] + (b1r*x[2] - b1i*x[3])*(x[2]**2 + x[3]**2) + (b2r*x[2] - b2i*x[3])*(x[0]**2 + x[1]**2) + (x[0]+x[1]+x[2]+x[3])**5])
def f4(x): 
    return np.array([ w2*x[0] + (b1r*x[3] + b1i*x[2])*(x[2]**2 + x[3]**2) + (b2r*x[3] + b2i*x[2])*(x[0]**2 + x[1]**2) + (x[0]+x[1]+x[2]+x[3])**5])

all_fun = [f1,f2,f3,f4]
# def f1(x):
#     w = 1
#     a = -0.5
#     b = 0.1
#     return np.array([-w*x[1] + (a*x[0] - b*x[1])*(x[0]**2 + x[1]**2) + (x[0]+x[1])**5])
# def f2(x):
#     w = 1
#     a = -0.5
#     b = 0.1
#     return np.array([w*x[0] + (b*x[0] + a*x[1])*(x[0]**2 + x[1]**2) - (x[0]+x[1])**5])

In [9]:
def T2_mat(n):
    T2 = np.eye(n**2,dtype=int)
    rmidx = np.triu_indices(n,1)[1]*n + np.triu_indices(n,1)[0]
    T2 = np.delete(T2,rmidx,0)
    return T2


def S2_mat(n):
    S2 = np.eye(n**2,dtype=int)
    rmidx = np.triu_indices(n,1)[1]*n + np.triu_indices(n,1)[0]
    addidx = np.triu_indices(n,1)[0]*n + np.triu_indices(n,1)[1]
    S2[rmidx,addidx] = 1
    S2 = np.delete(S2,rmidx,1)
    return S2

def T3_mat(n):
    Bx3 = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)] # extracted from x \otimes Bx^2
    x_Bx2 = [(i,j,k) for i in range(n) for j in range(n) for k in range(j,n)] #  x \otimes Bx^2
    Bx3_idx = [x_Bx2.index(i) for i in Bx3]
    rmidx = list(set(range(len(x_Bx2)))-set(Bx3_idx))
    rmele = [x_Bx2[i] for i in rmidx]
    rmele = [tuple(sorted(i)) for i in rmele]
    rmidx_inBx3 = [Bx3.index(i) for i in rmele]
    T3 = np.eye(n*n*(n+1)//2,dtype=int)
    T3 = T3[Bx3_idx]
    return T3

def S3_mat(n):
    Bx3 = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)] # extracted from x \otimes Bx^2
    x_Bx2 = [(i,j,k) for i in range(n) for j in range(n) for k in range(j,n)] #  x \otimes Bx^2
    Bx3_idx = [x_Bx2.index(i) for i in Bx3]
    rmidx = list(set(range(len(x_Bx2)))-set(Bx3_idx))
    rmele = [x_Bx2[i] for i in rmidx]
    rmele = [tuple(sorted(i)) for i in rmele]
    rmidx_inBx3 = [Bx3.index(i) for i in rmele]
    S3 = np.eye(n*n*(n+1)//2,dtype=int)
    S3 = S3[:,Bx3_idx]
    S3[rmidx,rmidx_inBx3] = 1
    return S3

def Trissian(f_test, x0):
    """
    This function calculates the 3rd order derivative of a function f: R^n -> R
    input: 
        f_test is the function
        x0 where the 3rd order want to be calcuated
    return: 3-D matrix
    """
    Trissian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 0.001
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        #print(xp1)
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        #print(xp2)
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        #print(xm1)
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        #print(xm2)
        Trissian[i] = (-nd.Hessian(f_test)(xp2) + 8*nd.Hessian(f_test)(xp1)- 8*nd.Hessian(f_test)(xm1) + nd.Hessian(f_test)(xm2))/(12*h)
    return Trissian

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{u}^3 B_u^3 + O(u^4)$$

In [10]:
import numdifftools as nd
x = np.array([0,0,0,0],dtype=float) #Should explicitly assign dtype
J_c = nd.Jacobian(fun)(x)
pd.DataFrame(J_c)

,0,1,2,3
0,0.000000,-0.310000,0.000000,0.000000
1,0.310000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,-0.810000
3,0.000000,0.000000,0.810000,0.000000


In [11]:
n_c = 4
J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
# display(pd.DataFrame(J_cbar))
C2 = -np.matmul(np.matmul(T2_mat(n_c),J_cbar),S2_mat(n_c))
display(pd.DataFrame(C2))

,0,1,2,3,4,5,6,7,8,9
0,-0.000000,0.620000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,-0.310000,-0.000000,-0.000000,-0.000000,0.310000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,-0.000000,-0.000000,-0.000000,0.810000,-0.000000,0.310000,-0.000000,-0.000000,-0.000000,-0.000000
3,-0.000000,-0.000000,-0.810000,-0.000000,-0.000000,-0.000000,0.310000,-0.000000,-0.000000,-0.000000
4,-0.000000,-0.620000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
5,-0.000000,-0.000000,-0.310000,-0.000000,-0.000000,-0.000000,0.810000,-0.000000,-0.000000,-0.000000
6,-0.000000,-0.000000,-0.000000,-0.310000,-0.000000,-0.810000,-0.000000,-0.000000,-0.000000,-0.000000
7,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,1.620000,-0.000000
8,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.810000,-0.000000,0.810000
9,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-1.620000,-0.000000


In [12]:
Uu2 = np.zeros((n_c,n_c*(n_c+1)//2))
for i in range(n_c):
    hes = nd.Hessian(all_fun[i])(x) # The original Hessian of each f in all_fun
    hes[np.triu_indices(n_c,1)] *= 2  # double each element above the main diagonal
    Uu2[i] = hes[np.triu_indices(n_c)] # Keep upper triangular part
display(pd.DataFrame(Uu2))

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
n = n_c
import math
Gain = []
W3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
for i in W3_idx:
    val = 1
    for j in range(n_c):
        val *= math.factorial(i.count(j))
    Gain.append(val)

    
Gain = np.diag(1/np.array(Gain))

In [14]:
Gain.shape

(20, 20)

In [15]:
n = n_c
Uu3 = np.zeros((n,(n*(n+1)*(n+2)//6)))
# Uu3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
for i in range(n):
    t = Trissian(all_fun[i], x)
    Uu3[i] = [t[j] for j in W3_idx]
Uu3 = Uu3.dot(Gain)
display(pd.DataFrame(Uu3))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.510000,-0.150000,0.000000,0.000000,-0.510000,0.000000,0.000000,1.840000,-0.000000,1.840000,-0.150000,0.000000,0.000000,3.220000,-0.000000,3.220000,0.000000,0.000000,0.000000,0.000000
1,0.150000,-0.510000,0.000000,0.000000,0.150000,-0.000000,-0.000000,-3.220000,-0.000000,-3.220000,-0.510000,0.000000,0.000000,1.840000,-0.000000,1.840000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-0.818000,0.775000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.818000,0.775000,-0.000000,-0.000000,0.000000,0.152000,2.821000,0.152000,2.821000
3,-0.000000,-0.000000,-0.775000,-0.818000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.775000,-0.818000,0.000000,-0.000000,-0.000000,-2.821000,0.152000,-2.821000,0.152000


In [16]:
import scipy as sp
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2
display(pd.DataFrame(R_2n))

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000


In [17]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uu3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
display(pd.DataFrame(Ny3))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.510000,-0.150000,0.000000,0.000000,-0.510000,0.000000,0.000000,1.840000,-0.000000,1.840000,-0.150000,0.000000,0.000000,3.220000,-0.000000,3.220000,0.000000,0.000000,0.000000,0.000000
1,0.150000,-0.510000,0.000000,0.000000,0.150000,-0.000000,-0.000000,-3.220000,-0.000000,-3.220000,-0.510000,0.000000,0.000000,1.840000,-0.000000,1.840000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-0.818000,0.775000,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.818000,0.775000,-0.000000,-0.000000,0.000000,0.152000,2.821000,0.152000,2.821000
3,-0.000000,-0.000000,-0.775000,-0.818000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.775000,-0.818000,0.000000,-0.000000,-0.000000,-2.821000,0.152000,-2.821000,0.152000


### Calculate $H_{3n}$ for $y = z + H_{3n} B_z^3$ in $J_c H_{3n} + H_{3n} C_3 + N_y^3 = R_{3n}$

In [18]:
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])

# display(pd.DataFrame(C3))
# Lambda3 = np.kron(np.eye(n_c*(n_c+1)*(n_c+2)//6),J_c) + np.kron(C3.T, np.eye(n_c))
Lambda3 = np.kron(J_c, np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c), C3.T) # This is row vec

# display(pd.DataFrame(Lambda3))

In [19]:
L3 = np.zeros((80,8))
L3[[0,4,21,30],[0,0,0,0]] = 1 # a1_R
L3[[7,9,33,35],[2,2,2,2]] = 1 # a2_R
L3[[56,58,77,79],[4,4,4,4]] = 1 # b1_R
L3[[42,51,63,72],[6,6,6,6]] = 1 # b2_R

L3[[1,10],[1,1]] = -1
L3[[20,24],[1,1]] = 1   #a1_I
L3[[13,15],[3,3]] = -1
L3[[27,29],[3,3]] = 1    #a2_I
L3[[57,59],[5,5]] = -1
L3[[76,78],[5,5]] = 1    #b1_I
L3[[43,52],[7,7]] = -1
L3[[62,71],[7,7]] = 1    #b2_I

In [24]:
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
L_inv = np.linalg.inv(L)

# U_zero_rows = np.argsort(np.max(abs(U),axis=1,keepdims=True),axis=0)[0:8]
U_zero_rows = np.where(abs(np.diag(U)) < 1e-12)[0]
L2 = L_inv[U_zero_rows,:]

theta = np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-0.51 ],
       [ 0.15 ],
       [ 1.84 ],
       [-3.22 ],
       [ 0.152],
       [-2.821],
       [-0.818],
       [-0.775]])

In [25]:
# w1 = 0.31
# w2 = 0.81
# a1r = -0.51
# a1i = 0.15
# a2r = 1.84
# a2i = -3.22
# b1r = 0.152
# b1i = -2.821
# b2r = -0.818
# b2i = -0.775

In [29]:
pd.DataFrame(U[U_zero_rows,:].T)
# np.where(abs(U[U_zero_rows,:].T) > 1e-10)

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
q, r = np.linalg.qr(Lambda3)
zero_rows = np.where(abs(np.diag(r)) < 1e-12)[0]
q_inv = q.T
q2 = q_inv[zero_rows,:]

theta =np.linalg.inv(q2.dot(L3)).dot(q2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-0.51 ],
       [ 0.15 ],
       [ 1.84 ],
       [-3.22 ],
       [ 0.152],
       [-2.821],
       [-0.818],
       [-0.775]])

$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)+(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

In [49]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# fig,ax = plt.subplots(figsize=(10,10))
# ax.spy(r, precision=0.000000001, markersize=6)
# plt.grid(True)

In [ ]:
a1r = -0.5
a1i = 0.1
a2r = 1.8
a2i = -3.2
b1r = 0.15
b1i = -2.8
b2r = -0.8
b2i = -0.8